<a href="https://colab.research.google.com/github/antonum/Redis-Workshops/blob/main/10-TriggersFunctions/10-Triggers%20and%20Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Triggers and Functions

![Redis](https://redis.com/wp-content/themes/wpx/assets/images/logo-redis.svg?auto=webp&quality=85,75&width=120)

The triggers and functions feature of Redis Stack and its documentation are currently in preview, and only available in Redis Stack 7.2 or later. Latest RedisStack is installed and used by default in this notebook (no additional actions required).

The triggers and functions feature of Redis Stack allows running JavaScript functions inside Redis. These functions can be executed on-demand, by an event-driven trigger, or by a stream processing trigger.

See documentation at: https://redis.io/docs/interact/programmability/triggers-and-functions/

In [51]:
# Install redis client
!pip install -q redis

### Install Redis Stack locally

Even if you are using Redis Cloud you might need to install Redis Stack just for the cli tool `redis-cli`.

In [33]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes


deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


gpg: cannot open '/dev/tty': No such device or address
curl: (23) Failed writing body


### Setup Redis connection
No changes needed if you are using in-notebook locally installed Redis Stack. If using Redis Cloud - enter your endpoint host, port and credentials here.

In [34]:
import redis
import os
REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")
#Replace values above with your own if using Redis Cloud instance
#REDIS_HOST="redis-17231.c228.us-central1-1.gce.cloud.redislabs.com"
#REDIS_PORT=17231
#REDIS_PASSWORD="0XKOePIFBCtuNvV6PhsXl3ysQY6XXXX"

#shortcut for redis-cli $REDIS_CONN command
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"

### Basic Test with Redis CLI

In [35]:
!redis-cli $REDIS_CONN PING
!redis-cli $REDIS_CONN SET hello world
!redis-cli $REDIS_CONN GET hello

PONG
OK
"world"


### Basic Test with Python client

In [36]:
r = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD)

r.get("hello")


b'world'

Create the function code. Code must start with the #!js prologue on a first line.

In [37]:
code = """#!js api_version=1.0 name=lib

redis.registerFunction('hello', ()=> {
  return 'Hello from redis!'
})
"""


## Create and execute function with redis-cli

In [38]:
with open("tmp.js", "w") as f:
    f.write(code)
!redis-cli $REDIS_CONN -x TFUNCTION LOAD REPLACE < ./tmp.js
!redis-cli $REDIS_CONN TFCALL lib.hello 0

OK
"Hello from redis!"


In [39]:
res = r.tfunction_load(code, replace=True)
print(res)
res = r.tfcall(lib_name="lib", func_name="hello")
print(res)

b'OK'
b'Hello from redis!'


# Triggers

Functions within Redis can respond to events using keyspace triggers. While the majority of these events are initiated by command invocations, they also include events that occur when a key expires or is removed from the database.

For the full list of supported events, please refer to the Redis keyspace notifications page.

The following code creates a new keyspace trigger that adds a new field to a new or updated hash with the latest update time.

In [40]:
trigger_code = """#!js api_version=1.0 name=myFirstLibrary

redis.registerKeySpaceTrigger("consumer", "", function(client, data){
    if (client.call("type", data.key) != "hash") {
        // key is not a has, do not touch it.
        return;
    }
    // get the current time in ms
    var curr_time = client.call("time")[0];
    // set '__last_updated__' with the current time value
    client.call('hset', data.key, '__Last_Updated__', curr_time);
});
"""
res = r.tfunction_load(trigger_code, replace=True)
print(res)

b'OK'


In [42]:
#create hash
!redis-cli $REDIS_CONN hset hash1 field1 1

#read hash
!redis-cli $REDIS_CONN hgetall hash1

(integer) 0
1) "field1"
2) "1"
3) "__Last_Updated__"
4) "1698104001"


In [50]:
#list functions
!redis-cli $REDIS_CONN TFUNCTION LIST WITHCODE

1)  1) "api_version"
    2) "1.0"
    3) "cluster_functions"
    4) (empty array)
    5) "code"
    6) "#!js api_version=1.0 name=myFirstLibrary\n\nredis.registerKeySpaceTrigger(\"consumer\", \"\", function(client, data){\n    if (client.call(\"type\", data.key) != \"hash\") {\n        // key is not a has, do not touch it.\n        return;\n    }\n    // get the current time in ms\n    var curr_time = client.call(\"time\")[0];\n    // set '__last_updated__' with the current time value\n    client.call('hset', data.key, '__Last_Updated__', curr_time);\n});\n"
    7) "configuration"
    8) (nil)
    9) "engine"
   10) "js"
   11) "functions"
   12) (empty array)
   13) "keyspace_triggers"
   14) 1) "consumer"
   15) "name"
   16) "myFirstLibrary"
   17) "pending_async_calls"
   18) (empty array)
   19) "pending_jobs"
   20) (integer) 0
   21) "stream_triggers"
   22) (empty array)
   23) "user"
   24) "default"
2)  1) "api_version"
    2) "1.0"
    3) "cluster_functions"
    4) (empty ar